In [53]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from datetime import datetime

In [64]:
def unpickle(file):
    with open(file, 'rb') as f:
        _dict = pickle.load(f, encoding='bytes')
    return _dict
cifar = []
for num in range(5):
    cifar.append(unpickle(r'C:\Users\matthew\Documents\ScienceFair2017-2018\PythonCode\NeuralNet\cifar-10_test\data_batch_{}'.format(num + 1)))
test = unpickle(r'C:\Users\matthew\Documents\ScienceFair2017-2018\PythonCode\NeuralNet\cifar-10_test\test_batch')
unshaped = np.stack([cifar[x][b'data'] for x in range(len(cifar))]).reshape(-1, 3, 32, 32)
data = np.empty((unshaped.shape[0], 32, 32, 3))
for i in range(data.shape[3]):
    data[:, :, :, i] = unshaped[:, i, :, :]
train_labels = np.stack([cifar[x][b'labels'] for x in range(len(cifar))]).reshape(50000)
test_data_unshaped = test[b'data'].reshape(-1, 3, 32, 32)
test_data = np.empty((test[b'data'].shape[0], 32, 32, 3))
for i in range(test_data.shape[3]):
    test_data[:, :, :, i] = test_data_unshaped[:, i, :, :]
test_labels = test[b'labels']

In [80]:
from tensorflow.python.training import moving_averages
from tensorflow.python.training import control_flow_ops
def _get_variable(name,
                  shape,
                  initializer,
                  weight_decay=0.0,
                  dtype='float',
                  trainable=True):
    "A little wrapper around tf.get_variable to do weight decay and add to"
    "resnet collection"
    if weight_decay > 0:
        regularizer = tf.contrib.layers.l2_regularizer(weight_decay)
    else:
        regularizer = None
    collections = [tf.GraphKeys.VARIABLES, 'resnet_variables']
    return tf.get_variable(name,
                           shape=shape,
                           initializer=initializer,
                           dtype=dtype,
                           regularizer=regularizer,
                           collections=collections,
                           trainable=trainable)

In [82]:
def batch_norm(x, n_out, phase_train):
    with tf.variable_scope('bn'):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]), name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]), name='gamma', trainable=True)
        #moving_mean = _get_variable('moving_mean', [n_out], initializer=tf.zeros_initializer, trainable=False)
        #moving_variance = _get_variable('moving_variance', [n_out], initializer=tf.ones_initializer, trainable=False)
        batch_mean, batch_var = tf.nn.moments(x, [0, 1, 2], name='moments')
        #update_moving_mean = moving_averages.assign_moving_average(moving_mean, batch_mean, .9997)
        #update_moving_variance = moving_averages.assign_moving_average(moving_variance, batch_variance, .9997)
        #tf.add_to_collection('resnet_update_ops', update_moving_mean)
        #tf.add_to_collection('resnet_update_ops', update_moving_variance)
        #mean, variance = control_flow_ops.cond(phase_train, lambda: (mean, variance), lambda: (moving_mean, moving_variance))
        #x = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 1e-3)
        #return x
        ema = tf.train.ExponentialMovingAverage(decay=0.9)
        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)
        mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed
X = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')
training = tf.placeholder(tf.bool, name='training')
with tf.name_scope('cnn'):
    with tf.name_scope('conv_1') and tf.variable_scope('conv1'):
        c1 = tf.layers.conv2d(X, filters=16, kernel_size=3, strides=[1, 1], name='c1', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c1_bn = batch_norm(c1, 16, training)
        c1_out = tf.nn.relu(c1_bn)
    with tf.name_scope('conv_2') and tf.variable_scope('conv2'):
        c2 = tf.layers.conv2d(c1_out, filters=16, kernel_size=3, strides=[1, 1], name='c2', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c2_bn = batch_norm(c2, 16, training)
        c2_out = tf.nn.relu(c2_bn)
        c3 = tf.layers.conv2d(c2_out, filters=16, kernel_size=3, strides=[1, 1], name='c3', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c3_bn = batch_norm(c3, 16, training)
        c3_out = tf.nn.relu(c3_bn + c1_out)
    with tf.name_scope('conv_3') and tf.variable_scope('conv3'):
        c4 = tf.layers.conv2d(c3_out, filters=32, kernel_size=3, strides=[2, 2], name='c4', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c4_bn = batch_norm(c4, 32, training)
        c4_out = tf.nn.relu(c4_bn)
        c5 = tf.layers.conv2d(c4_out, filters=32, kernel_size=3, strides=[1, 1], name='c5', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c5_bn = batch_norm(c5, 32, training)
        short1 = tf.layers.conv2d(c3_out, filters=32, kernel_size=1, strides=[2, 2], name='short1', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c5_out = tf.nn.relu(c5_bn + short1)
    with tf.name_scope('conv_4') and tf.variable_scope('conv4'):
        c6 = tf.layers.conv2d(c5_out, filters=64, kernel_size=3, strides=[2, 2], name='c6', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c6_bn = batch_norm(c6, 64, training)
        c6_out = tf.nn.relu(c6_bn)
        c7 = tf.layers.conv2d(c6_out, filters=64, kernel_size=3, strides=[1, 1], name='c7', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c7_bn = batch_norm(c7, 64, training)
        short2 = tf.layers.conv2d(c5_out, filters=64, kernel_size=1, strides=[2, 2], name='short2', padding='SAME', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
        c7_out = tf.nn.relu(c7_bn + short2)
    with tf.name_scope('fc') and tf.variable_scope('fc'):
        c7_flat = tf.reshape(c7_out, [-1, 4096])
        logits = tf.layers.dense(c7_flat, 10, name='fc', kernel_initializer=tf.contrib.layers.variance_scaling_initializer())
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
learning_rate = 0.1
with tf.name_scope('train'):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 10
batch_size = 100
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)
xentropy_summary = tf.summary.scalar('xentropy', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(50000 // batch_size):
            index = iteration * batch_size
            X_batch = data[index:index + batch_size, :, :, :]
            y_batch = train_labels[index:index + batch_size]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % 10 == 0:
                summary_str = xentropy_summary.eval(feed_dict={X: X_batch, y: y_batch, training: True})
                step = epoch * (50000 // batch_size) + iteration
                file_writer.add_summary(summary_str, step)
            #print(iteration)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, training: True})
        acc_test = accuracy.eval(feed_dict={X: test_data, y: test_labels, training: False})
        print(epoch, 'Train accuracy:', acc_train, 'Test accuracy', acc_test)
    save_path = saver.save(sess, './resnet20_test.ckpt')

0 Train accuracy: 0.61 Test accuracy 0.5577


KeyboardInterrupt: 

In [81]:
tf.reset_default_graph()